# Installing Packages

In [18]:
!pip install dataframe-image
!pip install pandas==1.5.3


In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/criminal-psychological-data/Criminal Psychological Dataset CSV.csv
/kaggle/input/national-survey-of-drug-use-and-health-20152019/NSDUH_2015-2019.csv


# Importing Libraries

In [20]:
from sklearn.model_selection import LeaveOneOut

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score, cross_val_predict

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
from matplotlib import interactive
import matplotlib.image as mpimg
from matplotlib.pyplot import figure

# For plotting the classification results
from mlxtend.plotting import plot_decision_regions
import xgboost as xgb
from IPython.display import Image  
from six import StringIO

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import seaborn as sns
%matplotlib inline
from sklearn.metrics import make_scorer, f1_score, accuracy_score, confusion_matrix

# Dataset Processing

In [21]:
#Original suicide dataset
#reading and preparing datasets
org_ds = pd.read_csv('/kaggle/input/criminal-psychological-data/Criminal Psychological Dataset CSV.csv')

#converting all columns to lowercase to standardize uss
org_ds.columns = org_ds.columns.str.lower()

#dataset without non-numeric unwanted fields like date, time, patient number, zero change values
numeric_ds = org_ds.drop(['cid','cjdid','staffid','idate','intstrt','instrta','suindt','stjail','pre30jl','pre180jl','cdob',
                          'chethn1','crace1','c23cmin','c23cmdt','ag1sdin','sdinj30','intend','intenda','editdate','d2bi2pma',
                          'd2bi2pmb','d2bi2pmc','c18pdmcl','c18pdmcp','c18pdmca','c18spgmc','anytxpm','datadmit','entpro',
                          'compl','onset1','neworrec','environ','course','worst','worst2','txhispsy','txdralc','hospspec',
                          'hospmed','othprob3','whowith','currdx','lifedes1','lifedes2','lifedes3','lifetx3','lifedes4',
                          'lifetx4','lifedes5','lifetx8','screen6a','s61b','s61bs','c17cmdt'] , axis = 1)

print("After removing unimportant fields " , len(numeric_ds.columns))

#converting categorical values to integer by One Hot Encoding
numeric_ds = pd.get_dummies(numeric_ds, columns = ['clive1','job1','majsup1','othoff','othdrug','qtother1',
                                                             'othtx1','resptx1','a38spgmc','a40spsu','c20spsu','f29spsu',
                                                             'diff1','diff2','diff3','onset2','new2','othprob1','othprob2',
                                                             'medvit','alcdrug','freetime','ruleout','lifetx1','lifeage2',
                                                             'lifetx2','lifetx5','lifedes6','lifetx6','lifedes7','lifetx7','lifedes8'])

print("After converting categorical fields " , len(numeric_ds.columns))

# replace field that's entirely space (or empty) with NaN
numeric_ds = numeric_ds.replace(r'^\s*$', np.nan, regex=True)
    
for (columnName, columnData) in numeric_ds.iteritems():
    numeric_ds[columnName] = pd.to_numeric(numeric_ds[columnName])
    numeric_ds[columnName].fillna(numeric_ds[columnName].mean())
    numeric_ds[columnName] = numeric_ds[columnName].replace(np.nan, numeric_ds[columnName].mean())
    numeric_ds[columnName].astype(str).astype(float)
    
#removing special characters from feature names added by One Hot Encoding
import re
# Change columns names ([LightGBM] Do not support special JSON characters in feature name. We were using LightGBM initially instead of CatBoost)
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in numeric_ds.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
numeric_ds = numeric_ds.rename(columns=new_names)

#important mix of features from anchor's top 19 and 12 features that also co-exist in the other dataset
importantFeatures_org = numeric_ds[['age1coc','mhsf28a','sidelf','gain44','age1tob','gs5a','age1mj','sattlf']]


threshold = 0.5  # Set the threshold value

# Convert float values to 0 or 1 based on the threshold
importantFeatures_org['age1coc'] = (importantFeatures_org['age1coc'] > threshold).astype(int)

importantFeatures_org.info()

After removing unimportant fields  860
After converting categorical fields  2337


/tmp/ipykernel_33/895485643.py:30: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for (columnName, columnData) in numeric_ds.iteritems():


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   age1coc  353 non-null    int64
 1   mhsf28a  353 non-null    int64
 2   sidelf   353 non-null    int64
 3   gain44   353 non-null    int64
 4   age1tob  353 non-null    int64
 5   gs5a     353 non-null    int64
 6   age1mj   353 non-null    int64
 7   sattlf   353 non-null    int64
dtypes: int64(8)
memory usage: 22.2 KB


/tmp/ipykernel_33/895485643.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  importantFeatures_org['age1coc'] = (importantFeatures_org['age1coc'] > threshold).astype(int)


In [22]:
#PROCESSED DATASET WITHOUT SUICIDE IDEATION
x = importantFeatures_org.drop(['sattlf'], axis=1) #X training features
y = importantFeatures_org.sattlf #y output features
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=3) # 80% training and 20% test

# Functions & Methods

In [23]:
#Creating Dataframe table for displaying of performance metrics for ensemble models
#required for table image creation
metrics_name = ["Accuracy(%):", "F1-Score:", "Precision:", "Sensitivity:",
               "AUC:","PPV:", "Logloss:" , "True +:", "True -:",
               "False +:", "False -:"]
metrics_val = [0,1,2,3,4,5,6,7,8,9,10]

table_df = pd.DataFrame()
table_df["Metrics"] = metrics_name

def gen_performance_metrics(classifier , YT , YP):
    TN, FP, FN, TP = confusion_matrix(YT, YP).ravel()
    metrics_val[0] = round(accuracy_score(YT, YP)*100,2)
    metrics_val[1] = round(f1_score(YT, YP, average='macro'),3)
    metrics_val[2] = round(precision_score(YT, YP, average='micro'),3)
    metrics_val[3] = round(recall_score(YT, YP, average='binary'),3)
    metrics_val[4] = round(metrics.roc_auc_score(YT, YP),3)
    metrics_val[5] = round(TP/(TP+FP),3)
    metrics_val[6] = round(log_loss(YT, YP, eps=1e-5, normalize=True, sample_weight=None, labels=None),3)
    metrics_val[7] = TP
    metrics_val[8] = TN
    metrics_val[9] = FP
    metrics_val[10] = FN
    table_df[classifier] = metrics_val
    print(table_df)

In [24]:
# Define columns to read, values that co-exist with previous dataset
columns_to_read = ['cocage','mhsuitry','suicthnk','mhsutk_u','cigtry','bkrob','mjage','suictry']

# Read the CSV file and select only the specified columns
important_features_new = pd.read_csv('/kaggle/input/national-survey-of-drug-use-and-health-20152019/NSDUH_2015-2019.csv', 
                                     usecols=columns_to_read)

# Map values to be similar to previous dataset. 1 for yes and 0 for no
important_features_new['suicthnk'] = important_features_new['suicthnk'].map({1:1,2: 0})
important_features_new['bkrob'] = important_features_new['bkrob'].map({1:1,2: 0})
important_features_new['bkrob'] = important_features_new['bkrob'].map({1:1,3: 1})
important_features_new['suictry'] = important_features_new['suictry'].map({1:1,2: 0})

#pre-processing
important_features_new['cocage'].replace(991, 0, inplace=True) #never used cocaine
important_features_new['cigtry'].replace(991, 0, inplace=True) #never used tobacco
important_features_new['mjage'].replace(991, 0, inplace=True) #never used marijuana
important_features_new['cocage'].replace(998, np.nan, inplace=True) #for blank labels
important_features_new['suicthnk'].replace(98, np.nan, inplace=True) #for blank labels
important_features_new['suicthnk'].replace(99, np.nan, inplace=True) #for blank labels
important_features_new['cigtry'].replace(994, np.nan, inplace=True) #for blank labels
important_features_new['bkrob'].replace(98, np.nan, inplace=True) #for blank labels
important_features_new['bkrob'].replace(94, np.nan, inplace=True) #for blank labels
important_features_new['suictry'].replace(98, np.nan, inplace=True) #for blank labels
important_features_new['suictry'].replace(99, np.nan, inplace=True) #for blank labels

#removing bad rows
important_features_new = important_features_new[important_features_new['cocage'] != 998]
important_features_new = important_features_new[important_features_new['cocage'] != 994]
important_features_new = important_features_new[important_features_new['cocage'] != 997]
important_features_new = important_features_new[important_features_new['suicthnk'] != 85]
important_features_new = important_features_new[important_features_new['suicthnk'] != 94]
important_features_new = important_features_new[important_features_new['suicthnk'] != 97]
important_features_new = important_features_new[important_features_new['cigtry'] != 985]
important_features_new = important_features_new[important_features_new['cigtry'] != 994]
important_features_new = important_features_new[important_features_new['bkrob'] != 85]
important_features_new = important_features_new[important_features_new['bkrob'] != 89]
important_features_new = important_features_new[important_features_new['bkrob'] != 97]
important_features_new = important_features_new[important_features_new['bkrob'] != 99]
important_features_new = important_features_new[important_features_new['suictry'] != 85]
important_features_new = important_features_new[important_features_new['suictry'] != 97]

# replace field that's entirely space (or empty) with NaN
important_features_new = important_features_new.replace(r'^\s*$', np.nan, regex=True)
    
for (columnName, columnData) in important_features_new.iteritems():
    important_features_new[columnName] = pd.to_numeric(important_features_new[columnName])
    important_features_new[columnName].fillna(important_features_new[columnName].mean())
    important_features_new[columnName] = important_features_new[columnName].replace(np.nan, important_features_new[columnName].mean())
    important_features_new[columnName].astype(str).astype(float)
    
    
# Assuming 'float_column' is the name of your float column
threshold = 0.5  # Set the threshold value

# Convert float values to 0 or 1 based on the threshold
important_features_new['bkrob'] = (important_features_new['bkrob'] > threshold).astype(int)
important_features_new['suicthnk'] = (important_features_new['suicthnk'] > threshold).astype(int)
important_features_new['mhsutk_u'] = (important_features_new['mhsutk_u'] > threshold).astype(int)
important_features_new['mhsuitry'] = (important_features_new['mhsuitry'] > threshold).astype(int)
important_features_new['suictry'] = (important_features_new['suictry'] > threshold).astype(int)


print("After taking relevant columns only" , len(important_features_new.columns))
# Display details about the columns in the DataFrame
important_features_new.info()
print(important_features_new['suictry'])

# Rename columns in the testing data to match the training data from previous dataset
important_features_new = important_features_new.rename(columns={'cigtry':'age1tob','mjage':'age1mj','cocage': 'age1coc','bkrob':'gs5a','suicthnk':'sidelf','mhsutk_u':'gain44','mhsuitry':'mhsf28a','suictry':'sattlf'})

# Reorder columns in the testing data to match the order expected by the model
important_features_new = important_features_new[['age1tob', 'age1mj', 'age1coc', 'gs5a', 'sidelf', 'gain44', 'mhsf28a', 'sattlf']]


xB = important_features_new.drop(['sattlf'], axis=1) #X training features
yB = important_features_new.sattlf #y output features
xTrainB, xTestB, yTrainB, yTestB = train_test_split(xB, yB, test_size=0.9, random_state=0)

/tmp/ipykernel_33/3166019888.py:46: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for (columnName, columnData) in important_features_new.iteritems():


After taking relevant columns only 8
<class 'pandas.core.frame.DataFrame'>
Int64Index: 281739 entries, 0 to 282767
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   cigtry    281739 non-null  float64
 1   mjage     281739 non-null  int64  
 2   cocage    281739 non-null  float64
 3   bkrob     281739 non-null  int64  
 4   suicthnk  281739 non-null  int64  
 5   suictry   281739 non-null  int64  
 6   mhsutk_u  281739 non-null  int64  
 7   mhsuitry  281739 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 19.3 MB
0         0
1         0
2         0
3         0
4         0
         ..
282763    0
282764    0
282765    0
282766    0
282767    0
Name: suictry, Length: 281739, dtype: int64


# Ensemble Method

In [25]:
#ensemble techniques for classifiers
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

# Create classifer objects for XGBoost and Random Forest models
xgbModel = xgb.XGBClassifier(objective="binary:logistic", learning_rate=0.3, 
                           booster='gbtree', max_depth=5, n_estimators=100, enable_categorical=False)
randomForestModel = RandomForestClassifier(n_estimators=500)

#Voting ensemble classifier of XGboost and Random Forest 
class_model = VotingClassifier(
    estimators=[('xgb', xgbModel),('ranFor',randomForestModel)], voting='hard')


# training all the model on the train dataset
xTrain = xTrain[['age1tob', 'age1mj', 'age1coc', 'gs5a', 'sidelf', 'gain44', 'mhsf28a']]
class_model.fit(xTrain, yTrain)
# predicting the output on the test dataset
xTest = xTest[['age1tob', 'age1mj', 'age1coc', 'gs5a', 'sidelf', 'gain44', 'mhsf28a']]
y_pred = class_model.predict(xTest)
xTestB = xTestB[['age1tob', 'age1mj', 'age1coc', 'gs5a', 'sidelf', 'gain44', 'mhsf28a']]
y_predB = class_model.predict(xTestB)

    
gen_performance_metrics("ENS" , yTest, y_pred)
gen_performance_metrics("ENS-B" , yTestB, y_predB)

         Metrics     ENS
0   Accuracy(%):  95.770
1      F1-Score:   0.900
2     Precision:   0.958
3   Sensitivity:   0.778
4           AUC:   0.881
5           PPV:   0.875
6       Logloss:   0.486
7        True +:   7.000
8        True -:  61.000
9       False +:   1.000
10      False -:   2.000
         Metrics     ENS       ENS-B
0   Accuracy(%):  95.770      99.560
1      F1-Score:   0.900       0.880
2     Precision:   0.958       0.996
3   Sensitivity:   0.778       1.000
4           AUC:   0.881       0.998
5           PPV:   0.875       0.617
6       Logloss:   0.486       0.051
7        True +:   7.000    1813.000
8        True -:  61.000  250627.000
9       False +:   1.000    1126.000
10      False -:   2.000       0.000
